In [2]:
import tsfresh
import tslearn
import pandas as pd
import numpy as np

# tsfresh example from docs



In [3]:
from tsfresh.examples import load_robot_execution_failures


from tsfresh.examples.robot_execution_failures import download_robot_execution_failures
download_robot_execution_failures()

timeseries, y = load_robot_execution_failures()

In [22]:
timeseries

,id,time,F_x,F_y,F_z,T_x,T_y,T_z
0,1,0,-1,-1,63,-3,-1,0
1,1,1,0,0,62,-3,-1,0
2,1,2,-1,-1,61,-3,0,0
3,1,3,-1,-1,63,-2,-1,0
4,1,4,-1,-1,63,-3,-1,0
...,...,...,...,...,...,...,...,...
1315,88,10,-10,2,39,-21,-24,5
1316,88,11,-11,2,38,-24,-22,6
1317,88,12,-12,3,23,-24,-24,5
1318,88,13,-13,4,26,-29,-27,5


In [5]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████| 48/48 [00:08<00:00,  5.98it/s]


In [6]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)
features_filtered = select_features(extracted_features, y)

C:\Users\sevi\anaconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['T_z__partial_autocorrelation__lag_7'
 'T_z__partial_autocorrelation__lag_8'
 'T_z__partial_autocorrelation__lag_9' ...
 'T_y__matrix_profile__feature_"median"__threshold_0.98'
 'T_y__matrix_profile__feature_"25"__threshold_0.98'
 'T_y__matrix_profile__feature_"75"__threshold_0.98'] did not have any finite values. Filling with zeros.
  warnings.warn(


In [7]:
from tsfresh import extract_relevant_features

features_filtered_direct = extract_relevant_features(timeseries, y, column_id='id', column_sort='time')

Feature Extraction: 100%|██████████| 48/48 [00:08<00:00,  5.93it/s]


In [8]:
features_filtered_direct

,F_x__value_count__value_-1,F_x__abs_energy,F_x__root_mean_square,T_y__absolute_maximum,F_x__mean_n_absolute_max__number_of_maxima_7,F_x__range_count__max_1__min_-1,F_y__abs_energy,F_y__root_mean_square,F_y__mean_n_absolute_max__number_of_maxima_7,T_y__variance,...,"F_y__cwt_coefficients__coeff_13__w_2__widths_(2, 5, 10, 20)","F_y__cwt_coefficients__coeff_14__w_5__widths_(2, 5, 10, 20)",T_y__lempel_ziv_complexity__bins_3,T_y__quantile__q_0.1,F_z__time_reversal_asymmetry_statistic__lag_1,F_x__quantile__q_0.2,F_y__quantile__q_0.7,"T_x__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0",T_z__large_standard_deviation__r_0.35000000000000003,T_z__quantile__q_0.9
1,14.0,14.0,0.966092,1.0,1.000000,15.0,13.0,0.930949,1.000000,0.222222,...,-0.310265,-0.751682,0.400000,-1.0,-5.960000e+02,-1.0,-1.0,0.000000,0.0,0.0
2,7.0,25.0,1.290994,5.0,1.571429,13.0,76.0,2.250926,3.000000,4.222222,...,-0.202951,0.057818,0.533333,-3.6,-6.803846e+02,-1.0,-1.0,0.000000,1.0,0.0
3,11.0,12.0,0.894427,5.0,1.000000,14.0,40.0,1.632993,2.142857,3.128889,...,0.539121,0.912474,0.533333,-4.0,-6.170000e+02,-1.0,0.0,0.000000,1.0,0.0
4,5.0,16.0,1.032796,6.0,1.285714,10.0,60.0,2.000000,2.428571,7.128889,...,-2.641390,-0.609735,0.533333,-4.6,3.426308e+03,-1.0,1.0,0.000000,0.0,0.0
5,9.0,17.0,1.064581,5.0,1.285714,13.0,46.0,1.751190,2.285714,4.160000,...,0.591927,0.072771,0.466667,-5.0,-2.609000e+03,-1.0,0.8,0.000000,0.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0.0,96833.0,80.346334,167.0,105.285714,0.0,42780.0,53.404120,71.428571,1563.528889,...,38.559593,71.641254,0.533333,36.4,-7.700628e+07,-105.0,66.8,64.000000,0.0,-1.4
85,0.0,1683.0,10.592450,14.0,13.714286,0.0,1523.0,10.076375,12.142857,14.755556,...,14.429645,16.349699,0.466667,1.0,-1.050785e+04,5.8,10.6,13.555556,0.0,-1.4
86,0.0,83497.0,74.608757,191.0,98.142857,0.0,21064.0,37.473546,47.714286,2788.595556,...,60.760842,71.095480,0.466667,19.6,-5.544922e+06,30.4,38.4,0.250000,0.0,2.8
87,0.0,1405437.0,306.097697,471.0,340.000000,0.0,308658.0,143.447551,157.285714,6415.715556,...,109.029954,173.699573,0.466667,272.6,-9.881845e+07,246.8,154.8,0.000000,0.0,43.2


# Try using UCR Dataset

In [9]:
from tsfresh import extract_features
# from tslearn.utils import load_time_series_txt


dataset = pd.read_table("UCRArchive_2018\Trace\Trace_TEST.tsv", header=None)
type(dataset)

pandas.core.frame.DataFrame

In [30]:
from tslearn.clustering import TimeSeriesKMeans

km = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5,random_state=0).fit(dataset)


# Format clusters into TSFresh preferred data format


In [50]:
# km.cluster_centers_[0]
# data = pd.DataFrame()
frames = []

for i in range(km.n_clusters):
    df = pd.DataFrame(km.cluster_centers_[i], columns=['x'])
    df['id'] = i
    frames.append(df)


result = pd.concat(frames)
result.head

<bound method NDFrame.head of              x  id
0     2.545455   0
1    -0.519348   0
2    -0.527836   0
3    -0.520405   0
4    -0.519483   0
..         ...  ..
272   0.655940   2
273   0.660174   2
274   0.658710   2
275   0.668273   2
276  15.000000   2

[831 rows x 2 columns]>

In [58]:
extracted_features = extract_features(result, column_id='id')
extracted_features.shape

Feature Extraction: 100%|██████████| 3/3 [00:05<00:00,  1.99s/it]


(3, 789)

In [59]:
extracted_features

,x__variance_larger_than_standard_deviation,x__has_duplicate_max,x__has_duplicate_min,x__has_duplicate,x__sum_values,x__abs_energy,x__mean_abs_change,x__mean_change,x__mean_second_derivative_central,x__median,...,x__permutation_entropy__dimension_6__tau_1,x__permutation_entropy__dimension_7__tau_1,x__query_similarity_count__query_None__threshold_0.0,"x__matrix_profile__feature_""min""__threshold_0.98","x__matrix_profile__feature_""max""__threshold_0.98","x__matrix_profile__feature_""mean""__threshold_0.98","x__matrix_profile__feature_""median""__threshold_0.98","x__matrix_profile__feature_""25""__threshold_0.98","x__matrix_profile__feature_""75""__threshold_0.98",x__mean_n_absolute_max__number_of_maxima_7
0,1.0,0.0,0.0,0.0,49.575758,2278.637544,0.195402,0.161177,0.089872,-0.167521,...,3.810631,4.090744,NaN,2.301108,15.083478,6.732279,4.454829,3.286701,11.169796,7.568689
1,1.0,0.0,0.0,0.0,83.583333,6704.172099,0.316402,0.287540,0.150838,-0.036139,...,3.899462,4.313877,NaN,2.294942,15.674032,8.763448,9.835030,3.518775,13.231705,12.615184
2,1.0,0.0,0.0,0.0,17.645161,314.010342,0.077740,0.044764,0.032288,0.031890,...,3.834691,4.181701,NaN,2.284787,13.696806,3.594736,3.635057,2.884713,3.978087,3.082981


In [70]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
y = pd.Series([0,1,2])
x = result.loc[:, 'id']
impute(extracted_features)
features_filtered = select_features(extracted_features, y)


In [71]:
# features_filtered

In [73]:
from tsfresh import extract_relevant_features

features_filtered_direct = extract_relevant_features(result, x, column_id='id')

ValueError: The following ids are in y but are missing inside the time series container: {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205,

# end


In [10]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,266,267,268,269,270,271,272,273,274,275
0,3,-1.296736,-1.333738,-1.349496,-1.318587,-1.307479,-1.255202,-1.273145,-1.254287,-1.298989,...,0.817590,0.851303,0.782615,0.801945,0.821274,0.810207,0.822209,0.819276,0.816343,0.794338
1,1,0.564214,0.545116,0.542047,0.563157,0.620198,0.581974,0.598603,0.578583,0.612213,...,0.568564,0.577551,0.620840,0.520834,0.568744,0.626330,0.530965,0.571332,0.586189,0.590860
2,3,-2.100779,-2.088201,-2.025100,-2.100239,-2.086126,-2.030557,-2.057606,-2.094286,-2.019413,...,0.589671,0.496224,0.537520,0.558377,0.547514,0.571221,0.542142,0.542945,0.543747,0.543874
3,2,0.738442,0.728175,0.757858,0.822484,0.734091,0.760751,0.811335,0.802686,0.728860,...,0.760828,0.709781,0.750877,0.738374,0.706462,0.769492,0.705494,0.738047,0.764293,0.757955
4,1,0.637616,0.585078,0.623133,0.629319,0.641032,0.644209,0.620274,0.629437,0.693167,...,0.619496,0.588804,0.622431,0.619396,0.637752,0.640945,0.647132,0.635257,0.621143,0.648951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,-1.371061,-1.401310,-1.360921,-1.361051,-1.399245,-1.391050,-1.395285,-1.336950,-1.379701,...,0.842057,0.793177,0.806116,0.808740,0.811363,0.749296,0.770993,0.775988,0.830288,0.800406
96,2,0.713223,0.693700,0.668581,0.709537,0.760939,0.737603,0.671763,0.652010,0.704283,...,0.625502,0.765896,0.699376,0.749527,0.610622,0.718855,0.713569,0.711667,0.722169,0.755220
97,2,0.806887,0.811921,0.807797,0.843080,0.829992,0.783205,0.812043,0.804756,0.840494,...,0.746681,0.701849,0.733755,0.752613,0.821207,0.796000,0.772949,0.761621,0.834557,0.752477
98,3,-1.934068,-1.966324,-1.951075,-1.985240,-1.952251,-1.964241,-1.997795,-1.978881,-1.874253,...,0.557510,0.571033,0.601152,0.515953,0.548378,0.577351,0.527907,0.542919,0.588434,0.534182


In [18]:
from tsfresh.utilities.dataframe_functions import roll_time_series

rolled = roll_time_series(dataset, column_id=0)



Rolling: 100%|██████████| 29/29 [00:05<00:00,  5.04it/s]


In [23]:
# rolled

Feature Extraction: 100%|██████████| 50/50 [00:12<00:00,  4.05it/s]


In [19]:
extracted_features

,1__variance_larger_than_standard_deviation,1__has_duplicate_max,1__has_duplicate_min,1__has_duplicate,1__sum_values,1__abs_energy,1__mean_abs_change,1__mean_change,1__mean_second_derivative_central,1__median,...,275__permutation_entropy__dimension_6__tau_1,275__permutation_entropy__dimension_7__tau_1,275__query_similarity_count__query_None__threshold_0.0,"275__matrix_profile__feature_""min""__threshold_0.98","275__matrix_profile__feature_""max""__threshold_0.98","275__matrix_profile__feature_""mean""__threshold_0.98","275__matrix_profile__feature_""median""__threshold_0.98","275__matrix_profile__feature_""25""__threshold_0.98","275__matrix_profile__feature_""75""__threshold_0.98",275__mean_n_absolute_max__number_of_maxima_7
1,0.0,0.0,0.0,0.0,14.796418,9.169316,0.053285,0.000387,-0.003314,0.613461,...,2.944439,2.890372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.630235
2,0.0,0.0,0.0,0.0,21.526881,16.048064,0.052153,0.002444,0.001646,0.742871,...,3.178054,3.135494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.789437
3,0.0,0.0,0.0,0.0,-47.583116,82.665006,0.317123,-0.023605,0.004635,-1.710097,...,3.135494,3.091042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.775935
4,0.0,0.0,0.0,0.0,-31.326165,53.204026,0.294754,-0.001489,0.025681,-1.603391,...,2.639057,2.564949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790837


maybe there is a pattern in extracted features not seen by DTW


In [28]:
# extracted_features
# y = dataset.iloc[:,0]
# type(y)
# y

In [ ]:
from tsfresh import extract_relevant_features

features_filtered_direct = extract_relevant_features(dataset, y, column_id='id')

AssertionError: y needs to be a pandas.Series, received type: <class 'int'>.